In [1]:
import pandas as pd
import numpy as np

In [2]:
df1 = pd.read_csv('2024_apr30_orders.csv')
df1['title'] = df1['title'].str.lstrip()
df1['title'] = df1['title'].str.rstrip()
df1['title_lower'] = df1['title'].str.lower()
print(df1.shape)

temp_df = pd.read_csv('product_feed.csv')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&gt;', '>')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&amp;', '&')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&#039;', "'")
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&gt;', '>')
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&amp;', '&')
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&#039;', "'")
temp_df.rename(columns={'TITLE': 'title_cat'}, inplace=True)
# temp_stock_df = temp_df['title_cat'].str.split('\(|\)', expand=True).iloc[:,[0,1]]
temp_stock_df = temp_df['title_cat'].str.split('\(Size:', expand=True)
temp_df['title_cat'] = temp_stock_df[0]
temp_df['title_cat'] = temp_df['title_cat'].str.lstrip()
temp_df['title_cat'] = temp_df['title_cat'].str.rstrip()
temp_df.drop_duplicates(subset=['title_cat'], inplace=True)
temp_df['title_lower'] = temp_df['title_cat'].str.lower()
temp_df = temp_df[['title_cat', 'title_lower', 'PRODUCT TYPE']]

<>:16: SyntaxWarning: invalid escape sequence '\('
<>:16: SyntaxWarning: invalid escape sequence '\('
C:\Users\retai\AppData\Local\Temp\ipykernel_7720\702489313.py:16: SyntaxWarning: invalid escape sequence '\('
  temp_stock_df = temp_df['title_cat'].str.split('\(Size:', expand=True)


(60960, 23)


In [3]:
df1['customs_description_old'] = df1['customs_description']

In [4]:
df1 = df1.merge(temp_df, on='title_lower', how='left')
df1['customs_description'] = df1['PRODUCT TYPE']
df1.drop(['PRODUCT TYPE'], axis=1, inplace=True)
df1.shape

(60960, 25)

In [5]:
print(df1['customs_description'].isnull().sum())
df1['customs_description'] = df1['customs_description'].fillna(df1['customs_description_old'])
df1.drop(['customs_description_old'], axis=1, inplace=True)
print(df1['customs_description'].isnull().sum())

14936
1258


In [6]:
df1['customs_description'].value_counts()

customs_description
Mens > Leather Jackets > Biker Styles       4399
Ladies > Leather Jackets > Biker Styles     3692
Ladies > Handbags                           2871
January Sale 2021                           2856
Ladies > Deals for HER > Handbags           2707
                                            ... 
Handbags > Saddle Bag                          2
Valetine's Deals 2024                          2
Ladies                                         2
Valentine's Day 2023                           1
Home & Living > Leather Magazine Baskets       1
Name: count, Length: 138, dtype: int64

In [7]:
df1['customs_description'] = df1['customs_description'].replace(np.NaN, '(No assigned category)')
print(df1['customs_description'].isnull().sum())

0


In [8]:
df2 = df1['customs_description'].str.split(' > ', expand=True)
df1.drop(['Category', 'F_Cat', 'S_Cat', 'T_Cat'], axis=1, inplace=True)
df2.columns = ['F_Cat', 'S_Cat', 'T_Cat']
df1 = df1.join(df2)
df1['Category'] = df1['F_Cat'] + ' > ' + df1['S_Cat']

In [9]:
df1.to_csv('2024_apr30_orders fixed_categories.csv')

In [10]:
temp_df[temp_df['title_cat'] == 'Ashwood Leather Crossbody Bag Navy: CB-3 (63014)']

,title_cat,title_lower,PRODUCT TYPE
8476,Ashwood Leather Crossbody Bag Navy: CB-3 (63014),ashwood leather crossbody bag navy: cb-3 (63014),Ladies > Leather Handbags


In [11]:
df1 = pd.read_csv('2024_apr30_orders.csv')
df1['title'] = df1['title'].str.lstrip()
df1['title'] = df1['title'].str.rstrip()
df1.shape

(60960, 22)

In [12]:
df1_notnull = df1[df1['customs_description'].notnull()]
df1_null = df1[df1['customs_description'].isna()]
df1_notnull.shape, df1_null.shape, len(df1_null.isna())

((58089, 22), (2871, 22), 2871)

In [13]:
56557 + 2628

59185

In [14]:
temp_df = pd.read_csv('product_feed.csv')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&gt;', '>')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&amp;', '&')
temp_df['PRODUCT TYPE'] = temp_df['PRODUCT TYPE'].str.replace('&#039;', "'")
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&gt;', '>')
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&amp;', '&')
temp_df['TITLE'] = temp_df['TITLE'].str.replace('&#039;', "'")
temp_df.rename(columns={'TITLE': 'title'}, inplace=True)
temp_stock_df = temp_df['title'].str.split('\(|\)', expand=True).iloc[:,[0,1]]
temp_df['title'] = temp_stock_df[0]
temp_df['title'] = temp_df['title'].str.lstrip()
temp_df['title'] = temp_df['title'].str.rstrip()
temp_df.drop_duplicates(subset=['title'], inplace=True)
temp_df = temp_df[['title', 'PRODUCT TYPE']]

<>:9: SyntaxWarning: invalid escape sequence '\('
<>:9: SyntaxWarning: invalid escape sequence '\('
C:\Users\retai\AppData\Local\Temp\ipykernel_7720\526015662.py:9: SyntaxWarning: invalid escape sequence '\('
  temp_stock_df = temp_df['title'].str.split('\(|\)', expand=True).iloc[:,[0,1]]


In [15]:
# temp_df.applymap(list)['title'].apply(pd.Series).stack().unique()

In [16]:
df1_null = df1_null.merge(temp_df, on='title', how='left')
df1_null['customs_description'] = df1_null['PRODUCT TYPE']
df1_null.drop(['PRODUCT TYPE'], axis=1, inplace=True)
df1_null.shape

(2871, 22)

In [17]:
df1 = pd.concat([df1_notnull, df1_null], ignore_index=True)
df1.drop_duplicates(inplace=True)
df1.shape

(60960, 22)

In [18]:
df2 = df1['customs_description'].str.split(' > ', expand=True)
df1.drop(['Category', 'F_Cat', 'S_Cat', 'T_Cat'], axis=1, inplace=True)
df2.columns = ['F_Cat', 'S_Cat', 'T_Cat']
df1 = df1.join(df2)
df1['Category'] = df1['F_Cat'] + ' > ' + df1['S_Cat']

In [19]:
print(df1['customs_description'].isnull().sum())

1366


In [20]:
df1.to_csv('2024_apr30_orders fixed_categories.csv', index=False)